# The Battle of Neighborhoods - Week 2

## Introduction/Business Problem

I has been living on the west side of the City of Toronto in Canada for the past few years. I love the neighborhood, mainly because of all the great amenities and other types of venues that exist in the neighborhood, such as local food stores, pharmacies, parks, grad schools and so on. However, my work locations have always been in downtown Toronto. So I’m thinking of moving to downtown or somewhere closer to downtown area. In order to make the decision, I want to understand if downtown has a similar or better neighborhoods compared to the west side of Toronto.

Areas that I'm interested needs to meet the following criteria:

- A neighborhood with an average to above average total population
- Above average populations of 25–40-year-old male and female professionals
- A high concentration of the population having secondary education
- Average to above average median net household incomes

## Describe the Data

The necessary information needed to make the moving decision will come from the following sources:

1). 
[City of Toronto Neighborhood Profiles](https://www.toronto.ca/city-government/data-research-maps/neighbourhoods-communities/neighbourhood-profiles/) for providing an overview of the neighborhoods in Toronto. 

[City of Toronto Open Data Catalogue](https://open.toronto.ca/#8c732154-5012-9afe-d0cd-ba3ffc813d5a) : for population data based on the Census of Population that was held across Canada in 2016. Information includes population, age and sex, type of dwelling, families, households and marital status, language, income, immigration and ethnocultural diversity, housing, Aboriginal peoples, education, labour, journey to work, language of work and mobility and migration, as measured in the census program.  [CSV File](https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv)

[City of Toronto Neighborhood Shapes](https://open.toronto.ca/dataset/neighbourhoods/) for mapping : GeoJSON File based on [CKAN API](https://docs.ckan.org/en/latest/api/index.html)

2). [Wikipedia for Toronto Neighborhood Borough Designation](https://en.wikipedia.org/wiki/List_of_city-designated_neighbourhoods_in_Toronto) : Each of the 140 social planning neighborhoods of Toronto reside within a defined borough. While the City of Toronto is a singular municipality, the 140 neighborhoods are still grouped into six distinct boroughs.

3).[Foursquare API](https://developer.foursquare.com/) to collect information on other venues/competitors in the neighborhoods of Toronto

In [91]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [219]:
#Data for Neighbourhood Number, Population and private dwellings:
df_torpop = pd.read_csv("https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv",encoding='latin1')

#drop columns Category, Topic, Data Source
df_torpop = df_torpop.drop('Category',1)
df_torpop = df_torpop.drop('Topic',1)
df_torpop = df_torpop.drop('Data Source',1)

#only keep selected rows
df_torpop = df_torpop.drop(index=df_torpop.index.difference([0, 2,7,20,21,22,41,42,43,74,1723,2354]))
df_torpop.index.names = [None]
#Transpose rows and columns
df_torpop = df_torpop.transpose()

#adjust column headers
df_torpop.columns = df_torpop.iloc[0]

df_torpop = df_torpop[2:]
df_torpop.index.names = [None]
#df_torpop.index.names = [None]
#df_torpop = df_torpop.set_index('Characteristic')
df_torpop.head()


Characteristic,Neighbourhood Number,"Population, 2016",Population density per square kilometre,Male: 25 to 29 years,Male: 30 to 34 years,Male: 35 to 39 years,Female: 25 to 29 years,Female: 30 to 34 years,Female: 35 to 39 years,Average household size,"University certificate, diploma or degree at bachelor level or above",After-tax income: Average amount ($)
Agincourt North,129,"29,113","3,929",1015,835,680,1005,935,775,3.16,4240,"26,955"
Agincourt South-Malvern West,128,"23,757","3,034",1045,820,625,975,835,715,2.88,4615,"27,928"
Alderwood,20,"12,054","2,435",355,410,455,350,430,450,2.6,1980,"39,159"
Annex,95,"30,526","10,863",2080,1610,1055,2265,1675,1040,1.8,12640,"80,138"
Banbury-Don Mills,42,"27,695","2,775",645,735,735,745,860,895,2.23,8060,"51,874"


In [220]:
print (df_torpop.dtypes)

Characteristic
Neighbourhood Number                                                        object
Population, 2016                                                            object
Population density per square kilometre                                     object
Male: 25 to 29 years                                                        object
Male: 30 to 34 years                                                        object
Male: 35 to 39 years                                                        object
Female: 25 to 29 years                                                      object
Female: 30 to 34 years                                                      object
Female: 35 to 39 years                                                      object
 Average household size                                                     object
    University certificate, diploma or degree at bachelor level or above    object
  After-tax income: Average amount ($)                                  

In [221]:

#Clean up the table
#Convert columns to integer values and get the total population for Male and Female
df_torpop['Male: 25 to 29 years'] = df_torpop['Male: 25 to 29 years'].str.replace(',','').astype(int)
df_torpop['Male: 30 to 34 years'] = df_torpop['Male: 30 to 34 years'].str.replace(',','').astype(int)
df_torpop['Male: 35 to 39 years'] = df_torpop['Male: 35 to 39 years'].str.replace(',','').astype(int)
sum_column = df_torpop["Male: 25 to 29 years"] + df_torpop["Male: 30 to 34 years"]+df_torpop["Male: 35 to 39 years"]
df_torpop["Male"] = sum_column

df_torpop['Female: 25 to 29 years'] = df_torpop['Female: 25 to 29 years'].str.replace(',','').astype(int)
df_torpop['Female: 30 to 34 years'] = df_torpop['Female: 30 to 34 years'].str.replace(',','').astype(int)
df_torpop['Female: 35 to 39 years'] = df_torpop['Female: 35 to 39 years'].str.replace(',','').astype(int)
sum_column = df_torpop["Female: 25 to 29 years"] + df_torpop["Female: 30 to 34 years"]+df_torpop["Female: 35 to 39 years"]
df_torpop["Female"] = sum_column

df_torpop = df_torpop.drop('Male: 25 to 29 years',1)
df_torpop = df_torpop.drop('Male: 30 to 34 years',1)
df_torpop = df_torpop.drop('Male: 35 to 39 years',1)
df_torpop = df_torpop.drop('Female: 25 to 29 years',1)
df_torpop = df_torpop.drop('Female: 30 to 34 years',1)
df_torpop = df_torpop.drop('Female: 35 to 39 years',1)

df_torpop = df_torpop.rename(columns={'Characteristic': 'Neighbourhood'})

df_torpop.head()

Characteristic,Neighbourhood Number,"Population, 2016",Population density per square kilometre,Average household size,"University certificate, diploma or degree at bachelor level or above",After-tax income: Average amount ($),Male,Female
Agincourt North,129,"29,113","3,929",3.16,4240,"26,955",2530,2715
Agincourt South-Malvern West,128,"23,757","3,034",2.88,4615,"27,928",2490,2525
Alderwood,20,"12,054","2,435",2.6,1980,"39,159",1220,1230
Annex,95,"30,526","10,863",1.8,12640,"80,138",4745,4980
Banbury-Don Mills,42,"27,695","2,775",2.23,8060,"51,874",2115,2500


In [222]:
print(df_torpop.dtypes)
df_torpop.shape

Characteristic
Neighbourhood Number                                                        object
Population, 2016                                                            object
Population density per square kilometre                                     object
 Average household size                                                     object
    University certificate, diploma or degree at bachelor level or above    object
  After-tax income: Average amount ($)                                      object
Male                                                                         int64
Female                                                                       int64
dtype: object


(140, 8)

In [223]:
#Rename Column Names
df_torpop = df_torpop.rename(columns={'Population density per square kilometre':'Population density/sk',' Average household size':'Average household size','    University certificate, diploma or degree at bachelor level or above': 'Higher Education','  After-tax income: Average amount ($)':'After-tax income'})
df_torpop['Population density/sk'] = df_torpop['Population density/sk'].str.replace(',','').astype(int)
df_torpop['Average household size'] = df_torpop['Average household size'].str.replace(',','').astype(float)
df_torpop['Higher Education'] = df_torpop['Higher Education'].str.replace(',','').astype(int)
df_torpop['After-tax income'] = df_torpop['After-tax income'].str.replace(',','').astype(int)
df_torpop['Neighbourhood Number'] = df_torpop['Neighbourhood Number'].astype(int)

print (df_torpop.dtypes)

Characteristic
Neighbourhood Number        int64
Population, 2016           object
Population density/sk       int64
Average household size    float64
Higher Education            int64
After-tax income            int64
Male                        int64
Female                      int64
dtype: object


In [224]:
import urllib
import json
import pandas as pd

# Get the dataset metadata by passing package_id to the package_search endpoint
# For example, to retrieve the metadata for this dataset:

url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
params = { "id": "4def3f65-2a65-4a4f-83c4-b2a4aed72d46"}
response = urllib.request.urlopen(url, data=bytes(json.dumps(params), encoding="utf-8"))
package = json.loads(response.read())
print(package["result"])

# Get the data by passing the resource_id to the datastore_search endpoint
# See https://docs.ckan.org/en/latest/maintaining/datastore.html for detailed parameters options
# For example, to retrieve the data content for the first resource in the datastore:

for idx, resource in enumerate(package["result"]["resources"]):
    if resource["datastore_active"]:
        url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/datastore_search"
        p = { "id": resource["id"] }
        r = urllib.request.urlopen(url, data=bytes(json.dumps(p), encoding="utf-8"))
        data = json.loads(r.read())
        df_neighborhood = pd.DataFrame(data["result"]["records"])
        break
df_neighborhood

{'license_title': 'Open Government Licence – Toronto', 'owner_unit': None, 'relationships_as_object': [], 'topics': 'Locations and mapping', 'owner_email': 'opendata@toronto.ca', 'excerpt': 'Boundaries of Toronto Neighbourhoods.', 'private': False, 'owner_division': 'Social Development, Finance & Administration', 'num_tags': 9, 'id': '4def3f65-2a65-4a4f-83c4-b2a4aed72d46', 'metadata_created': '2019-07-23T17:47:38.364157', 'refresh_rate': 'As available', 'title': 'Neighbourhoods', 'license_url': 'https://open.toronto.ca/open-data-license/', 'state': 'active', 'information_url': 'http://www.toronto.ca/demographics/neighbourhoods.htm', 'license_id': 'open-government-licence-toronto', 'type': 'dataset', 'resources': [{'cache_last_updated': None, 'package_id': '4def3f65-2a65-4a4f-83c4-b2a4aed72d46', 'datastore_active': True, 'id': 'a083c865-6d60-4d1d-b6c6-b0c8a85f9c15', 'size': None, 'format': 'GeoJSON', 'state': 'active', 'hash': '', 'description': '', 'is_preview': True, 'last_modified': 

,AREA_ATTR_ID,AREA_DESC,AREA_ID,AREA_LONG_CODE,AREA_NAME,AREA_SHORT_CODE,LATITUDE,LONGITUDE,OBJECTID,PARENT_AREA_ID,Shape__Area,Shape__Length,X,Y,_id,geometry
0,25926662,Wychwood (94),25886861,094,Wychwood (94),094,43.676919,-79.425515,16491505,49885,3.217960e+06,7515.779658,None,None,5181,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4359..."
1,25926663,Yonge-Eglinton (100),25886820,100,Yonge-Eglinton (100),100,43.704689,-79.403590,16491521,49885,3.160334e+06,7872.021074,None,None,5182,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4109..."
2,25926664,Yonge-St.Clair (97),25886834,097,Yonge-St.Clair (97),097,43.687859,-79.397871,16491537,49885,2.222464e+06,8130.411276,None,None,5183,"{""type"": ""Polygon"", ""coordinates"": [[[-79.3911..."
3,25926665,York University Heights (27),25886593,027,York University Heights (27),027,43.765736,-79.488883,16491553,49885,2.541821e+07,25632.335242,None,None,5184,"{""type"": ""Polygon"", ""coordinates"": [[[-79.5052..."
4,25926666,Yorkdale-Glen Park (31),25886688,031,Yorkdale-Glen Park (31),031,43.714672,-79.457108,16491569,49885,1.156669e+07,13953.408098,None,None,5185,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4396..."
5,25926667,Lambton Baby Point (114),25886404,114,Lambton Baby Point (114),114,43.657420,-79.496045,16491585,49885,3.406857e+06,11181.788316,None,None,5186,"{""type"": ""Polygon"", ""coordinates"": [[[-79.5055..."
6,25926668,Lansing-Westgate (38),25886627,038,Lansing-Westgate (38),038,43.754271,-79.424748,16491601,49885,1.025436e+07,15391.584157,None,None,5187,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4399..."
7,25926669,Lawrence Park North (105),25886808,105,Lawrence Park North (105),105,43.730060,-79.403978,16491617,49885,4.388577e+06,9051.488042,None,None,5188,"{""type"": ""Polygon"", ""coordinates"": [[[-79.3900..."
8,25926670,Lawrence Park South (103),25886812,103,Lawrence Park South (103),103,43.717212,-79.406039,16491633,49885,6.211341e+06,13530.370002,None,None,5189,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4109..."
9,25926671,Leaside-Bennington (56),25886328,056,Leaside-Bennington (56),056,43.703797,-79.366072,16491649,49885,9.161864e+06,20531.730658,None,None,5190,"{""type"": ""Polygon"", ""coordinates"": [[[-79.3774..."


In [225]:

print(df_neighborhood.dtypes)
df_neighborhood.shape

AREA_ATTR_ID         int64
AREA_DESC           object
AREA_ID              int64
AREA_LONG_CODE      object
AREA_NAME           object
AREA_SHORT_CODE     object
LATITUDE           float64
LONGITUDE          float64
OBJECTID             int64
PARENT_AREA_ID       int64
Shape__Area        float64
Shape__Length      float64
X                   object
Y                   object
_id                  int64
geometry            object
dtype: object


(100, 16)

In [226]:
#df_neighborhood = df_neighborhood.drop(df_neighborhood.columns[['AREA_ATTR_ID', 'AREA_DESC', 'AREA_ID', 'AREA_LONG_CODE', 'OBJECTID', 'PARENT_AREA_ID', 'X', 'Y', '_id', 'geometry']], axis=1)
df_neighborhood = df_neighborhood.drop('AREA_ATTR_ID',1)
df_neighborhood = df_neighborhood.drop('AREA_DESC',1)
df_neighborhood = df_neighborhood.drop('AREA_ID',1)
df_neighborhood = df_neighborhood.drop('OBJECTID',1)
df_neighborhood = df_neighborhood.drop('X',1)
df_neighborhood = df_neighborhood.drop('Y',1)
df_neighborhood = df_neighborhood.drop('_id',1)
df_neighborhood = df_neighborhood.drop('geometry',1)
df_neighborhood = df_neighborhood.drop('AREA_SHORT_CODE',1)

df_neighborhood.head()

,AREA_LONG_CODE,AREA_NAME,LATITUDE,LONGITUDE,PARENT_AREA_ID,Shape__Area,Shape__Length
0,094,Wychwood (94),43.676919,-79.425515,49885,3.217960e+06,7515.779658
1,100,Yonge-Eglinton (100),43.704689,-79.403590,49885,3.160334e+06,7872.021074
2,097,Yonge-St.Clair (97),43.687859,-79.397871,49885,2.222464e+06,8130.411276
3,027,York University Heights (27),43.765736,-79.488883,49885,2.541821e+07,25632.335242
4,031,Yorkdale-Glen Park (31),43.714672,-79.457108,49885,1.156669e+07,13953.408098


In [227]:
df_neighborhood['AREA_LONG_CODE'] = df_neighborhood['AREA_LONG_CODE'].astype(int)

In [228]:

df_neighborhood = df_neighborhood.rename(columns={'AREA_LONG_CODE': 'Neighbourhood Number'})
df_neighborhood.head()

,Neighbourhood Number,AREA_NAME,LATITUDE,LONGITUDE,PARENT_AREA_ID,Shape__Area,Shape__Length
0,94,Wychwood (94),43.676919,-79.425515,49885,3.217960e+06,7515.779658
1,100,Yonge-Eglinton (100),43.704689,-79.403590,49885,3.160334e+06,7872.021074
2,97,Yonge-St.Clair (97),43.687859,-79.397871,49885,2.222464e+06,8130.411276
3,27,York University Heights (27),43.765736,-79.488883,49885,2.541821e+07,25632.335242
4,31,Yorkdale-Glen Park (31),43.714672,-79.457108,49885,1.156669e+07,13953.408098


In [229]:
toronto_data = pd.merge(df_torpop,df_neighborhood, on = ['Neighbourhood Number'],how='inner')
col = list(toronto_data.columns)

In [230]:
col1 = [col[8]] + col[:8] + col[9:]
toronto_data = toronto_data[col1]

In [231]:
toronto_data

,AREA_NAME,Neighbourhood Number,"Population, 2016",Population density/sk,Average household size,Higher Education,After-tax income,Male,Female,LATITUDE,LONGITUDE,PARENT_AREA_ID,Shape__Area,Shape__Length
0,Agincourt North (129),129,"29,113",3929,3.16,4240,26955,2530,2715,43.805441,-79.266712,49885,1.395145e+07,17159.740667
1,Agincourt South-Malvern West (128),128,"23,757",3034,2.88,4615,27928,2490,2525,43.788658,-79.265612,49885,1.511736e+07,21320.849547
2,Alderwood (20),20,"12,054",2435,2.60,1980,39159,1220,1230,43.604937,-79.541611,49885,9.502180e+06,12667.013917
3,Annex (95),95,"30,526",10863,1.80,12640,80138,4745,4980,43.671585,-79.404001,49885,5.337192e+06,10513.883143
4,Banbury-Don Mills (42),42,"27,695",2775,2.23,8060,51874,2115,2500,43.737657,-79.349718,49885,1.924897e+07,25141.572290
5,Bathurst Manor (34),34,"15,873",3377,2.56,4030,37927,1530,1670,43.764813,-79.456055,49885,9.133974e+06,20183.925841
6,Bay Street Corridor (76),76,"25,797",14097,1.70,11660,43427,4820,4910,43.657511,-79.385721,49885,3.459075e+06,9594.336045
7,Bayview Village (52),52,"21,396",4195,2.22,7910,41440,2920,3175,43.776361,-79.377117,49885,9.902533e+06,13169.020187
8,Bayview Woods-Steeles (49),49,"13,154",3240,2.70,3640,38196,920,1005,43.796802,-79.382118,49885,7.853671e+06,11439.166125
9,Bedford Park-Nortown (39),39,"23,236",4209,2.68,7350,85678,1445,1905,43.731486,-79.420227,49885,1.057626e+07,17755.857560


In [107]:
print(toronto_data.dtypes)
toronto_data.shape


AREA_NAME                  object
Neighbourhood Number        int64
Population, 2016           object
Population density/sk       int64
Average household size    float64
Higher Education            int64
After-tax income            int64
Male                        int64
Female                      int64
LATITUDE                  float64
LONGITUDE                 float64
PARENT_AREA_ID              int64
Shape__Area               float64
Shape__Length             float64
dtype: object


(100, 14)

In [108]:
# Import the required libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [109]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#  Get the requested page
page = requests.get(wiki_url)
# Get the text data from the response object
data = page.text
# Parse the page
soup = BeautifulSoup(data)
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XrRJvgpAEKcAAI@-EI0AAAAQ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":955414546,"wgRevisionId":955414546,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

In [110]:
# Extract the table tag
wiki_table = soup.find('table')
wiki_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>East Y

In [111]:
# Extract the table rows
table_row = wiki_table.find_all('tr')
table_row

[<tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighborhood
 </th></tr>, <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>
 </td></tr>, <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>
 </td></tr>, <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>, <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>, <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>, <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>, <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>, <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>
 </td></tr>, <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue
 </td></tr>, <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>, <tr>
 <td>M2B
 </td>
 <td>Not assigned
 </td>
 <td>
 </td></tr>, <tr>
 <td>M3B
 </td>
 <td>North York
 </td>
 <td>Do

In [112]:
# Get the data from each table row

dataframe = []
for row in table_row:
    table_data = row.find_all('td')
    data = [i.text.rstrip() for i in table_data]
#     print(data)
    dataframe.append(data)

dataframe 
del dataframe[0]
dataframe

[['M1A', 'Not assigned', ''],
 ['M2A', 'Not assigned', ''],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M8A', 'Not assigned', ''],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M2B', 'Not assigned', ''],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', ''],
 ['M8B', 'Not assigned', ''],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'],
 ['M2C', 'Not assigned', ''],
 ['M3C', 'North York', 'Don Mills'],
 ['M4C', 'East York', 'Woodbine 

In [147]:
# Declare labels to assign to the dataframe
labels = ['Postcode', 'Borough', 'Neighborhood']

In [148]:
# Create pandas dataframe and load it with data
wiki_table_df = pd.DataFrame.from_records(dataframe, columns=labels)

In [149]:
# Remove the data cells with 'Not assigned' value in Borough
wiki_table_df = wiki_table_df[wiki_table_df.Borough != 'Not assigned']

In [150]:
wiki_table_df

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [151]:
#wdf = wiki_table_df.copy()
#wdf.Neighborhood[4].split(',')
path = "http://cocl.us/Geospatial_data"

wiki_final_df = pd.read_csv(path)
wiki_final_df.columns = ['Postcode', 'Latitude', 'Longitude']
wiki_final_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [153]:
#pd.melt(wdf, id_vars=['Postcode','Borough'], value_vars=['Neighborhood'])
toronto_data = pd.merge(wiki_table_df, wiki_final_df, on = ['Postcode'], how = 'inner')
toronto_data.head()

toronto_data.to_excel(r'D:\Study\Applied Data Science\Capstone Project\toronto_data.xlsx', index = None, header=True, encoding = 'utf8-8')

In [154]:
toronto_data


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [121]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [122]:
address = 'Toronto, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6534817, -79.3839347.


In [123]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Using Foursquare data to explore and cluster the neighborhoods in Toronto:

In [157]:
Analyze_data = pd.DataFrame()
Borough=['Downtown Toronto','Etobicoke']
#Analyze_data = toronto_data[toronto_data['Borough'] == 'Downtown Toronto' & toronto_data['Borough'] == 'Etobicoke'].reset_index(drop=True)
toronto_data = toronto_data[toronto_data.Borough.isin(Borough)]
#gapminder.year.isin(years)
Analyze_data = toronto_data
#gapminder_years= gapminder[gapminder.year.isin(years)]
#gapminder_years.shape


CLIENT_ID = '0L10D11EWECHVBUJ0G2DLNPE1V4T2FTUIGPOBVUFAEYCLNCY' # your Foursquare ID
CLIENT_SECRET = '3RHFMICGU02OJ5B4FBTY4N4NL4OFFT0LL2AL1XC4A1AC2RMB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
#Get the top 100 venues around this neighborhood with radius of 1000 meters
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

In [158]:
Analyze_data

,Postcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
17,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564


In [161]:
# define a function to repeat the process in downtown
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [163]:
Analyze_venues = getNearbyVenues(names=Analyze_data['Neighborhood'],
                                   latitudes=Analyze_data['Latitude'],
                                   longitudes=Analyze_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Islington Avenue
Garden District, Ryerson
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
St. James Town
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Westmount
Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
New Toronto, Mimico South, Humber Bay Shores
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
Rosedale
Stn A PO Boxes
Alderwood, Long Branch
Northwest
St. James Town, Cabbagetown
First Canadian Place, Underground city
The Kingsway

In [164]:
print(Analyze_venues.shape)
Analyze_venues.head()

(1270, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [165]:
Analyze_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alderwood, Long Branch",9,9,9,9,9,9
Berczy Park,55,55,55,55,55,55
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,63,63,63,63,63,63
Christie,17,17,17,17,17,17
Church and Wellesley,74,74,74,74,74,74
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood",8,8,8,8,8,8
"First Canadian Place, Underground city",100,100,100,100,100,100


In [166]:
print('There are {} uniques categories.'.format(len(Analyze_venues['Venue Category'].unique())))

There are 218 uniques categories.


### One-Hot-Encoding to devise the Feature set

In [167]:
# one hot encoding
Analyze_onehot = pd.get_dummies(Analyze_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Analyze_onehot['Neighborhood'] = Analyze_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Analyze_onehot.columns[-1]] + list(Analyze_onehot.columns[:-1])
Analyze_onehot = Analyze_onehot[fixed_columns]

Analyze_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,

In [168]:
# mean of frquence
Analyze_grouped = Analyze_onehot.groupby('Neighborhood').mean().reset_index()
Analyze_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [169]:

##print topmost venues
num_top_venues = 5

for hood in Analyze_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Analyze_grouped[Analyze_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.22
1      Pharmacy  0.11
2           Pub  0.11
3  Skating Rink  0.11
4           Gym  0.11


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2          Restaurant  0.04
3  Seafood Restaurant  0.04
4              Bakery  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3     Boat or Ferry  0.06
4   Harbor / Marina  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1  Italian Restaurant  0.08
2                Café  0.05
3      Sandwich Place  0.05
4        Burger Joint  0.03


----Christie----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12
3    Candy Store  0.06
4     Baby Store  0.06


----Church and Wel

In [171]:
## Create a dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Analyze_grouped['Neighborhood']

for ind in np.arange(Analyze_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Analyze_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alderwood, Long Branch",Pizza Place,Gym,Pool,Skating Rink,Sandwich Place,Pub,Coffee Shop,Pharmacy,Colombian Restaurant,Deli / Bodega
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Beer Bar,Bakery,Café,Cheese Shop,Japanese Restaurant,Shopping Mall
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Coffee Shop,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Plane
3,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Thai Restaurant,Japanese Restaurant,Salad Place,Ice Cream Shop,Bubble Tea Shop
4,Christie,Grocery Store,Café,Park,Candy Store,Diner,Italian Restaurant,Baby Store,Athletics & Sports,Restaurant,Coffee Shop
5,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Hotel,Pub,Café,Men's Store,Mediterranean Restaurant,Yoga Studio
6,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Italian Restaurant,Seafood Restaurant,Deli / Bodega,Japanese Restaurant
7,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Pet Store,Cosmetics Shop,Liquor Store,Pizza Place,Coffee Shop,Beer Store,Café,Convenience Store,Discount Store,Distribution Center
8,"First Canadian Place, Underground city",Coffee Shop,Café,Restaurant,Gym,Hotel,Japanese Restaurant,Salad Place,Steakhouse,American Restaurant,Deli / Bodega
9,"Garden District, Ryerson",Clothing Store,Coffee Shop,Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Café,Italian Restaurant,Japanese Restaurant,Bubble Tea Shop,Theater


## Clustering neighbors

In [178]:
# set number of clusters
kclusters = 5

Analyze_grouped_clustering = Analyze_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Analyze_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:29]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 4, 1,
       1, 1, 1, 0, 1, 1, 1], dtype=int32)

In [185]:
Analyze_merged = Analyze_data
Analyze_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
17,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564


In [181]:
neighborhoods_venues_sorted.shape

(29, 11)

In [182]:
Analyze_merged.dtypes

Postcode         object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [183]:
neighborhoods_venues_sorted.dtypes

Neighborhood              object
1st Most Common Venue     object
2nd Most Common Venue     object
3rd Most Common Venue     object
4th Most Common Venue     object
5th Most Common Venue     object
6th Most Common Venue     object
7th Most Common Venue     object
8th Most Common Venue     object
9th Most Common Venue     object
10th Most Common Venue    object
dtype: object

In [208]:
# create a new dataframe
Analyze_merged = neighborhoods_venues_sorted

# add clustering labels
Analyze_merged['Cluster_Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Analyze_merged = Analyze_merged.join(Analyze_data.set_index('Neighborhood'), on='Neighborhood')

Analyze_merged.head() # check the last columns!

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster_Labels,Postcode,Borough,Latitude,Longitude
0,"Alderwood, Long Branch",Pizza Place,Gym,Pool,Skating Rink,Sandwich Place,Pub,Coffee Shop,Pharmacy,Colombian Restaurant,Deli / Bodega,1,M8W,Etobicoke,43.602414,-79.543484
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Beer Bar,Bakery,Café,Cheese Shop,Japanese Restaurant,Shopping Mall,1,M5E,Downtown Toronto,43.644771,-79.373306
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Coffee Shop,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Plane,1,M5V,Downtown Toronto,43.628947,-79.394420
3,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Thai Restaurant,Japanese Restaurant,Salad Place,Ice Cream Shop,Bubble Tea Shop,1,M5G,Downtown Toronto,43.657952,-79.387383
4,Christie,Grocery Store,Café,Park,Candy Store,Diner,Italian Restaurant,Baby Store,Athletics & Sports,Restaurant,Coffee Shop,1,M6G,Downtown Toronto,43.669542,-79.422564


In [209]:
col=list(Analyze_merged.columns)
col1 = col[12:14] + [col[0]] + col[14:16] + [col[11]] + col[1:11]
Analyze_merged = Analyze_merged[col1]

In [210]:
Analyze_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,1,Pizza Place,Gym,Pool,Skating Rink,Sandwich Place,Pub,Coffee Shop,Pharmacy,Colombian Restaurant,Deli / Bodega
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Beer Bar,Bakery,Café,Cheese Shop,Japanese Restaurant,Shopping Mall
2,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,1,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Coffee Shop,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Plane
3,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Thai Restaurant,Japanese Restaurant,Salad Place,Ice Cream Shop,Bubble Tea Shop
4,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Candy Store,Diner,Italian Restaurant,Baby Store,Athletics & Sports,Restaurant,Coffee Shop


In [211]:
# map visualization
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Analyze_merged['Latitude'], Analyze_merged['Longitude'], Analyze_merged['Neighborhood'], Analyze_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [213]:
#Cluster 1
Analyze_merged.loc[Analyze_merged['Cluster_Labels'] == 0, Analyze_merged.columns[[2] + list(range(5, Analyze_merged.shape[1]))]]

,Neighborhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,"The Kingsway, Montgomery Road, Old Mill North",0,River,Park,Smoke Shop,Dance Studio,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [214]:
#CLuster 2
Analyze_merged.loc[Analyze_merged['Cluster_Labels'] == 1, Analyze_merged.columns[[2] + list(range(5, Analyze_merged.shape[1]))]]

,Neighborhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alderwood, Long Branch",1,Pizza Place,Gym,Pool,Skating Rink,Sandwich Place,Pub,Coffee Shop,Pharmacy,Colombian Restaurant,Deli / Bodega
1,Berczy Park,1,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Beer Bar,Bakery,Café,Cheese Shop,Japanese Restaurant,Shopping Mall
2,"CN Tower, King and Spadina, Railway Lands, Har...",1,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Coffee Shop,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Plane
3,Central Bay Street,1,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Thai Restaurant,Japanese Restaurant,Salad Place,Ice Cream Shop,Bubble Tea Shop
4,Christie,1,Grocery Store,Café,Park,Candy Store,Diner,Italian Restaurant,Baby Store,Athletics & Sports,Restaurant,Coffee Shop
5,Church and Wellesley,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Hotel,Pub,Café,Men's Store,Mediterranean Restaurant,Yoga Studio
6,"Commerce Court, Victoria Hotel",1,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Italian Restaurant,Seafood Restaurant,Deli / Bodega,Japanese Restaurant
7,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",1,Pet Store,Cosmetics Shop,Liquor Store,Pizza Place,Coffee Shop,Beer Store,Café,Convenience Store,Discount Store,Distribution Center
8,"First Canadian Place, Underground city",1,Coffee Shop,Café,Restaurant,Gym,Hotel,Japanese Restaurant,Salad Place,Steakhouse,American Restaurant,Deli / Bodega
9,"Garden District, Ryerson",1,Clothing Store,Coffee Shop,Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Café,Italian Restaurant,Japanese Restaurant,Bubble Tea Shop,Theater


In [215]:
#CLuster 3
Analyze_merged.loc[Analyze_merged['Cluster_Labels'] == 2, Analyze_merged.columns[[2] + list(range(5, Analyze_merged.shape[1]))]]

,Neighborhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,"Old Mill South, King's Mill Park, Sunnylea, Hu...",2,Baseball Field,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [216]:
#CLuster 4
Analyze_merged.loc[Analyze_merged['Cluster_Labels'] == 3, Analyze_merged.columns[[2] + list(range(5, Analyze_merged.shape[1]))]]

,Neighborhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Northwest,3,Drugstore,Bar,Rental Car Location,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [217]:
#CLuster 5
Analyze_merged.loc[Analyze_merged['Cluster_Labels'] == 4, Analyze_merged.columns[[2] + list(range(5, Analyze_merged.shape[1]))]]

,Neighborhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Rosedale,4,Park,Playground,Trail,Women's Store,Dance Studio,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [232]:
AREA_NAME=['Stonegate-Queensway (16)','Waterfront Communities-The Island (77)', 'Niagara (82)' ]
#Analyze_data = toronto_data[toronto_data['Borough'] == 'Downtown Toronto' & toronto_data['Borough'] == 'Etobicoke'].reset_index(drop=True)
toronto_data = toronto_data[toronto_data.AREA_NAME.isin(AREA_NAME)]
#gapminder.year.isin(years)
toronto_data

,AREA_NAME,Neighbourhood Number,"Population, 2016",Population density/sk,Average household size,Higher Education,After-tax income,Male,Female,LATITUDE,LONGITUDE,PARENT_AREA_ID,Shape__Area,Shape__Length
47,Niagara (82),82,"31,180",10156,1.62,15755,54803,8860,8920,43.636681,-79.412420,49885,6.192651e+06,15209.513429
73,Stonegate-Queensway (16),16,"25,051",3199,2.40,7010,49002,2255,2390,43.635518,-79.501128,49885,1.518135e+07,16379.806316
82,Waterfront Communities-The Island (77),77,"65,913",8943,1.61,35675,54292,18180,16805,43.633880,-79.377202,49885,2.562977e+07,25382.706586


## Results and Discussion section

First of all, even though most of the neighborhoods in Etobicoke are in the same cluster as of neighborhoods in Downtown Toronto, the venues within the two Borough are pretty different. However, they do have same level of similarities as well.

We may discuss our results under below main perspectives.

First, most common venues in Downtown Toronto are cafes and restaurants and Parks. Parks and restaurants are also a pretty common venues in Etobicoke area. 

Second, although restaurant is a common venue in both Borough, the cuisines mights be very different, and further research need to be done on that. On the other hand, Etobicoke erea has some hardware stores that Downtown doesn't have. 

Third, 

## Conclusion


To sum up, the analysis above would support my moving decision from Etobicoke to Downtown Toronto.

From the population census perspective, Downtown Toronto has far more people with higher education, and more yonge people from age 25 to 40. The household income in Downtown is also a lot higher than in Etobicoke. However, the overall population in Downtown is higher than Etobicoke, so I may feel crowd if moving to Downtown.

From the venues perspective, Downtown has more cafes and restaurants than the area I live. 

Overall, Downtown is more suitbale for yonge professionals or people looking for more social life and likes the busy part of the city.


